In [ ]:
!pip install -r requirements.txt

In [ ]:
import os


In [5]:
from groq import Groq
client = Groq(
    api_key=os.environ['GROQ_API_KEY']
)

In [7]:
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": "Explain the importance of fast language models"
        }
    ],
    model="llama3-groq-8b-8192-tool-use-preview"
)

In [8]:
print(chat_completion.choices[0].message.content)

Fast language models are crucial in today's data-driven world because they enable efficient processing and analysis of vast amounts of text data. Here are some key reasons why they're important:

1. **Speed and Efficiency**: Fast language models can process and generate text at incredible speeds, making them ideal for applications where time is of the essence, such as real-time chatbots, search engines, and content generation tools.
2. **Scalability**: These models can handle large volumes of data, allowing them to be used in applications that require processing massive datasets, like data analytics and machine learning.
3. **Improved Accuracy**: By leveraging advanced algorithms and training on vast amounts of data, fast language models can achieve higher accuracy in understanding and generating text, leading to better performance in tasks like language translation and text summarization.
4. **Enhanced User Experience**: Fast language models can power applications that provide instant

## Tool Use with Groq

In [40]:
from groq import Groq
import json

client = Groq()
MODEL = 'llama3-groq-70b-8192-tool-use-preview'

def calculate(expression):
    """Evaluate a mathematical expression"""
    try:
        result = eval(expression)
        print(expression)
        return json.dumps({"result": result})
    except:
        return json.dumps({"error": "Invalid expression"})

def run_conversation(user_prompt):
    messages=[
        {
            "role": "system",
            "content": "You are a calculator assistant. Use the calculate function to perform mathematical operations and provide the results."
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]
    tools = [
        {
            "type": "function",
            "function": {
                "name": "calculate",
                "description": "Evaluate a mathematical expression",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "expression": {
                            "type": "string",
                            "description": "The mathematical expression to evaluate",
                        }
                    },
                    "required": ["expression"],
                },
            },
        }
    ]
    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "calculate": calculate,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            print(tool_call)
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                expression=function_args.get("expression")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content

user_prompt = "What is 25 x 4 + 10 - 9.11?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_47jp', function=Function(arguments='{"expression": "25*4+10-9.11"}', name='calculate'), type='function')
25*4+10-9.11
The result of 25 x 4 + 10 - 9.11 is 100.89.


In [41]:
user_prompt = "What is 6 squared?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_1x55', function=Function(arguments='{"expression": "6^2"}', name='calculate'), type='function')
6^2
6 squared is 36.


In [ ]:
!pip install duckduckgo_search
!pip install wikipedia
!pip install request

In [55]:
import json
import requests
import wikipedia
from duckduckgo_search import DDGS


def duckduckgo_search(query):
    """"Perform a DuckDuckGo search using the duckduckgo_search library"""
    print(f"using DDG for {query}")

    try:
        with DDGS() as ddgs:
            results = list(ddgs.text(query, max_results=3))

        if not results:
            return f"No results found for '{query}. the search may have been to specific or there might be no relevant information available."
        
        # formatted_results = "\n".join([f"-{result['title']}: {result["body"]}" for result in results])
        formatted_results = "\n".join([f"-{result['title']}: {result['body']}" for result in results])
        return f"Here are some search results for '{query}:\n{formatted_results}'"
    except Exception as e:
        return f"An error occurred while search for '{query}': {str(e)}"

def wikipedia_search(query):
    """Perform a Wikipedia search"""
    try:
        print(f"Using Wikipedia for {query}")
        page = wikipedia.page(query)
        summary = wikipedia.summary(query, sentences=2)
        return json.dumps({"title": page.title, "summary": summary, "url": page.url})
    except:
        return json.dumps({"error": "No Wikipedia page found for the query"})


def run_conversation(user_prompt):
    messages=[
        {
            "role": "system",
            "content": "You are a search assistant. Use the duckduckgo_search function to perform web \
            searches and the wikipedia_search function to find information on Wikipedia. if you don't \
            need either of these then just reply normally"
        },
        {
            "role": "user",
            "content": user_prompt,
        }
    ]

    tools = [
        {
            "type": "function",
            "function": {
                "name": "internet_search",
                "description": "Perform a DuckDuckGo search",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query"
                        }
                    },
                    "required": "query"
                }
            }
        },
        {
            "type": "function",
            "function": {
                "name": "wikipedia_search",
                "description": "Perform a Wikipedia search",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "query": {
                            "type": "string",
                            "description": "The search query"
                        }
                    },
                    "required": ["query"]
                },
            }
        }
    ]

    response = client.chat.completions.create(
        model=MODEL,
        messages=messages,
        tools=tools,
        tool_choice="auto",
        max_tokens=4096
    )

    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    if tool_calls:
        available_functions = {
            "internet_search": duckduckgo_search,
            "wikipedia_search": wikipedia_search,
        }
        messages.append(response_message)
        for tool_call in tool_calls:
            print(tool_call)
            function_name = tool_call.function.name
            function_to_call = available_functions[function_name]
            function_args = json.loads(tool_call.function.arguments)
            function_response = function_to_call(
                query=function_args.get("query")
            )
            messages.append(
                {
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": function_response,
                }
            )
        second_response = client.chat.completions.create(
            model=MODEL,
            messages=messages
        )
        return second_response.choices[0].message.content


user_prompt = "What is the capital of Italy and what's a famous landmark there?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_t9jv', function=Function(arguments='{"query": "capital of Italy"}', name='wikipedia_search'), type='function')
Using Wikipedia for capital of Italy
ChatCompletionMessageToolCall(id='call_n62b', function=Function(arguments='{"query": "famous landmark in Italy"}', name='wikipedia_search'), type='function')
Using Wikipedia for famous landmark in Italy
The capital of Italy is Rome. You can find more information about Rome [here](https://en.wikipedia.org/wiki/Rome). For famous landmarks, I recommend checking out the Colosseum or the Vatican Museums.


In [54]:
# user_prompt = "What just happened in Tel Aviv?"
# print(run_conversation(user_prompt))

user_prompt = "What are the latest news about Tel Aviv?"
print(run_conversation(user_prompt))

ChatCompletionMessageToolCall(id='call_zg7q', function=Function(arguments='{"query": "latest news about Tel Aviv"}', name='duckduckgo_search'), type='function')
using DDG for latest news about Tel Aviv
Here are the latest news about Tel Aviv:
1. "Explosion in Tel Aviv Kills at Least One" by The New York Times: An explosion in central Tel Aviv killed at least one person and injured several others, the Israeli authorities said early Friday morning. A preliminary investigation indicated that a falling object might have caused the blast.
2. "Tel Aviv, Israel, explosion near US Embassy branch office kills one" by an unknown source: An explosion near the US Embassy branch office in Tel Aviv that killed at least one person early Friday is being investigated as a possible drone attack, according to Israeli authorities.
3. "Israeli military says Tel Aviv blast apparently caused by drone" by Reuters: People walk down the stairs of a building damaged at the site of an explosion, amid the Israel-H

In [ ]:
# question = f"""
#             Create a 5 levels of thinking.
#             1st, dumb and stupid
#             2st, easy and not deep.
#             3nd, medium.
#             4rd, hard.
#             5th, complex and creative.
#             6th, mind-blowing, never heard such an idea, out of the world idea, 250iq idea.

#             include setp-by-step guide how to achive the best option.

#             Question: how to solve the traffic jams in the city?
#             Answer:
#         """

# content = llm.chat.completions.create(
#     model=modelName, 
#     messages=[
#         {
#             "role": "user",
#             "content": question
#         },
#     ],
#     # response_format={"type": "json_object"}
# )